In [ ]:
import json
import os


def count_files_with_figure_text(base_directory):
    count = 0
    # elementary3부터 elementary6까지 디렉토리 탐색
    for grade in range(3, 7):  # 3에서 6까지
        directory = os.path.join(base_directory, f"elementary{grade}")
        if os.path.isdir(directory):
            for filename in os.listdir(directory):
                if filename.endswith(".json"):
                    file_path = os.path.join(directory, filename)
                    # BOM을 자동으로 처리하기 위해 utf-8-sig로 파일을 엽니다.
                    try:
                        with open(file_path, "r", encoding="utf-8-sig") as file:
                            file_content = (
                                file.read().strip()
                            )  # 파일 내용 읽기 및 공백 제거

                            # 빈 파일은 건너뛰기
                            if not file_content:
                                print(f"Skipping empty file: {file_path}")
                                continue

                            data = json.loads(
                                file_content
                            )  # json.loads로 로드하여 오류 확인

                            # OCR_info에 figure_text가 존재하고, null이 아닌 항목이 있는지 확인
                            if any(
                                item.get("figure_text") not in [None, "null", ""]
                                for item in data.get("OCR_info", [])
                            ):
                                count += 1
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON in file: {file_path} - {e}")
                    except Exception as e:
                        print(f"Error reading file {file_path}: {e}")
    return count


# base_directory 설정
base_directory = "./images/training/y"
result = count_files_with_figure_text(base_directory)
print(f"figure_text가 있는 파일의 수: {result}")

Skipping empty file: ./images/training/y\elementary3\P3_1_03_25885_160996.json
figure_text가 있는 파일의 수: 1964


In [ ]:
import os
import json
from transformers import AutoProcessor, AutoModel
import torch
from PIL import Image

# KoCLIP 모델과 프로세서 로드
processor = AutoProcessor.from_pretrained("koclip/koclip-base-pt")
model = AutoModel.from_pretrained("koclip/koclip-base-pt")

# 텍스트 옵션
text_options = ["지도", "도형", "그래프", "표"]


def predict_image_text_relation(image_path, text_options):
    # 이미지 파일 열기
    image = Image.open(image_path)

    # 이미지 전처리 및 텍스트 전처리 (모델에 입력할 수 있도록 변환)
    inputs = processor(
        images=image, text=text_options, return_tensors="pt", padding=True
    )

    # 모델을 통해 이미지와 텍스트 간의 임베딩을 추출
    with torch.no_grad():
        outputs = model(**inputs)

    # 이미지와 텍스트 간의 유사도 (logits_per_image)
    logits_per_image = outputs.logits_per_image

    # 유사도 점수 중 가장 높은 값을 가진 텍스트 찾기
    similarity_scores = logits_per_image.squeeze()  # 배치 차원 제거
    max_similarity_idx = torch.argmax(similarity_scores).item()
    max_similarity_score = similarity_scores[max_similarity_idx].item()

    # 임계값 설정 (유사도 점수가 충분히 높은 경우에만 출력)
    threshold = 0.7  # 필요에 따라 조정 가능
    if max_similarity_score >= threshold:
        print(f"이미지와 가장 유사한 텍스트: '{text_options[max_similarity_idx]}'")
        print(f"유사도 점수: {max_similarity_score:.4f}")
        return True
    else:
        print(
            f"이미지가 지정된 텍스트와 충분히 유사하지 않습니다. 유사도 점수: {max_similarity_score:.4f}"
        )
        return False


def process_json_files(json_dir):
    # figure_text가 있는 파일들을 찾아서 처리
    figure_text_files = []

    # 디렉토리 내의 모든 JSON 파일 검사
    for root, _, files in os.walk(json_dir):
        for file in files:
            if file.endswith(".json"):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, "r", encoding="utf-8-sig") as f:
                        data = json.load(f)
                        # figure_text가 있는 경우만 필터링
                        for item in data.get("OCR_info", []):
                            if item.get("figure_text"):
                                figure_text_files.append(data["question_filename"])
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")

    return figure_text_files


# 폴더 내 JSON 파일들이 있는 디렉토리 경로 지정 (y 폴더)
json_base_dir = "./images/training/y"

# x 폴더 내 이미지 파일 경로
image_base_dir = "./images/training/x"

# figure_text가 있는 파일들의 이름 가져오기 (elementary3부터 elementary6까지)
for i in range(3, 7):
    json_dir = os.path.join(json_base_dir, f"elementary{i}")

    # figure_text가 있는 파일들의 이름 가져오기
    figure_text_files = process_json_files(json_dir)

    # 가져온 이미지 파일들을 처리
    for filename in figure_text_files:
        image_path = os.path.join(image_base_dir, f"elementary{i}", filename)
        print(f"Processing image: {image_path}")
        if predict_image_text_relation(image_path, text_options):
            print(f"유효한 이미지로 처리 완료: {image_path}")
        else:
            print(f"유효하지 않은 이미지로 제외: {image_path}")

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

# BLIP 모델과 프로세서 로드
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained(
    "Salesforce/blip-image-captioning-base"
)


def generate_caption(image_path):
    # 이미지 파일 열기
    image = Image.open(image_path)

    # 이미지 전처리 및 모델 입력 준비
    inputs = processor(images=image, return_tensors="pt")

    # 모델을 통해 이미지에 대한 캡션 생성
    out = model.generate(**inputs)

    # 생성된 캡션 디코딩
    caption = processor.decode(out[0], skip_special_tokens=True)

    print(f"Generated Caption: {caption}")


# 테스트할 이미지 파일 경로 지정
image_path = "./images/training/x/elementary3/P3_2_03_39052_153966.png"
generate_caption(image_path)

Generated Caption: a diagram showing the area of a circle
